Attempt to create a movie recommendation system, data acquired at: https://www.kaggle.com/datasets/victorsoeiro/dark-matter-tv-shows-and-movies,
title recommendation is based on the information regarding the titles.csv and credit recomendation is based on credits.csv

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
from random import randint

In [2]:
df1 = pd.read_csv('titles.csv')
df2 = pd.read_csv('credits.csv')

In [3]:
df_titles = df1.copy()
drop_titles = ['age_certification', 'seasons', 'imdb_id']
df_titles.drop(drop_titles, axis=1, inplace=True)

df_credits = df2.copy()
drop_credits = ['character']
df_credits.drop(drop_credits, axis=1, inplace=True)

In [4]:
genres_agg = df_titles['genres'].unique()

In [5]:
g = []
for genres in genres_agg:
    if(genres == '[]'):
        g.append('[]')
        continue

    genres_tmp = genres[1:-1]
    if(',' in genres_tmp):
        genres_tmp = genres_tmp.replace(' ', '').split(',')
        for genre in genres_tmp:
            if(genre not in g):
                g.append(genre)
    else:
        if genres_tmp not in g:
            g.append(genres_tmp)

In [6]:
genres_agg = df_titles['genres']
genres = g
matrix_ohe_genres = np.zeros(shape=(genres_agg.shape[0], len(genres)))

In [7]:
for i, row in enumerate(genres_agg):
    for j, column in enumerate(genres):
        if column in row:
            matrix_ohe_genres[i][j] = 1

In [8]:
dataframe_ohe_genres = pd.DataFrame(matrix_ohe_genres, columns=genres)

In [9]:
countries_agg = df_titles['production_countries'].unique()

In [10]:
c = []
for countries in countries_agg:
    if(countries == '[]'):
        c.append('[]')
        continue
    
    countries_tmp = countries[1:-1]
    if(',' in countries_tmp):
        countries_tmp = countries_tmp.replace(' ', '').split(',')
        for country in countries_tmp:
            if(country not in c):
                c.append(country)
    else:
        if countries_tmp not in c:
            c.append(countries_tmp)

In [11]:
countries_agg = df_titles['production_countries']
countries = c
matrix_ohe_countries = np.zeros(shape=(countries_agg.shape[0], len(c)))

In [12]:
for i, row in enumerate(countries_agg):
    for j, column in enumerate(countries):
        if column in row:
            matrix_ohe_countries[i][j] = 1

In [13]:
dataframe_ohe_countries = pd.DataFrame(matrix_ohe_countries, columns=countries)

In [14]:
encoder = OneHotEncoder()
scaler = MinMaxScaler()
cols_to_encode = ['type']
cols_to_scale = ['release_year', 'runtime', 'imdb_score', 'imdb_votes', 'tmdb_popularity', 'tmdb_score']

In [15]:
encoded_cols = pd.DataFrame((encoder.fit_transform(df_titles[cols_to_encode])).toarray(), columns=['MOVIE', 'SHOW'])
scaled_cols = pd.DataFrame(scaler.fit_transform(df_titles[cols_to_scale]), columns=sorted(cols_to_scale))

In [16]:
df_tmp = pd.merge(encoded_cols, scaled_cols, left_index=True, right_index=True, how='outer')
df_tmp = pd.merge(df_tmp, dataframe_ohe_countries, left_index=True, right_index=True, how='outer')
df_tmp = pd.merge(df_tmp, dataframe_ohe_genres, left_index=True, right_index=True, how='outer')
df_tmp = pd.merge(df_tmp, df_titles['id'], left_index=True, right_index=True, how='outer')
df_tmp.set_index('id', inplace=True)
df_titles_final = df_tmp.fillna(0)

In [17]:
df_credits = df_credits.set_index('id')
name_role_agg = (df_credits['name'] + df_credits['role'])
name_role_agg_uni = name_role_agg.unique()
matrix_ohe_name = np.zeros(shape=(df_titles_final.shape[0], len(name_role_agg_uni)))

In [18]:
for i, row in enumerate(df_titles_final.index):
    if row in name_role_agg.index:
        tmp_rows = name_role_agg[row]
        for tmp_row in tmp_rows:
            for j, column in enumerate(name_role_agg_uni):
                if(tmp_row == column):
                    matrix_ohe_name[i][j] = 1

In [19]:
df_credits_final = pd.DataFrame(matrix_ohe_name, columns=name_role_agg_uni)
df_credits_final = df_credits_final.set_index([list(df_titles_final.index)])

In [20]:
df_final = pd.merge(df_credits_final, df_titles_final, left_index=True, right_index=True, how='outer')

In [21]:
model_nn_titles = NearestNeighbors(metric='cosine', n_neighbors=10)
model_nn_titles.fit(df_titles_final.values)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [22]:
model_nn_credits = NearestNeighbors(metric='cosine', n_neighbors=10)
model_nn_credits.fit(df_credits_final.values)

NearestNeighbors(metric='cosine', n_neighbors=10)

In [23]:
def get_movie_rec(movie_title):
    i = df_titles[df_titles['title'] == movie_title]['id']
    distances, indices = model_nn_titles.kneighbors(df_titles_final.loc[i].values, n_neighbors=6)
    print('If you enjoyed '+ movie_title+ ' then we think you\'ll like:')
    for i in range(1, 6):
        title = df_titles.iloc[indices[0][i]]['title']
        ry = str(df_titles.iloc[indices[0][i]]['release_year'])
        print(title + " (" + ry + ")")

In [24]:
def get_credits_rec(movie_title):
    i = df_titles[df_titles['title'] == movie_title]['id']
    distances, indices = model_nn_credits.kneighbors((df_credits_final.loc[i]).values, n_neighbors=6)
    for j in range(1, 6):
        id_movie = (df_titles.iloc[indices[0][j]].id)
        
        if(i.values[0] not in list(df_credits.index)):
            break
        a = df_credits.loc[i]
        b = df_credits.loc[id_movie]
        
        idx = (b.index[0])
        
        a.set_index('person_id', inplace=True)
        b.set_index('person_id', inplace=True)
        
        c =  pd.merge(a, b, left_index=True, right_index=True, how='inner')

        rec_title = (df_titles[df_titles['id'] == idx]['title'].values[0])
        rec_release = str(df_titles[df_titles['id'] == idx]['release_year'].values[0])
        if(c.shape[0] != 0):
            print('If you enjoyed ' + movie_title + ' for: ' + ", ".join(str(x) for x in list(c.name_x)) + ', they also appear in \"' + rec_title + '\" (' + rec_release + ')')

In [31]:
i = randint(0, 1021)

movie_title = df_titles.iloc[i].title
get_movie_rec(movie_title)
get_credits_rec(movie_title)

If you enjoyed Terror Tales then we think you'll like:
Parts Unknown (2018)
Creeping Crawling (2012)
The Passing (2011)
A Zombie Croc: Evil Has Been Summoned (2015)
Krampus: The Reckoning (2015)
If you enjoyed Terror Tales for: Christopher Showerman, they also appear in "Lady Bug" (2017)
If you enjoyed Terror Tales for: Laurene Landon, Yan Birch, they also appear in "Agramon's Gate" (2019)
If you enjoyed Terror Tales for: Felissa Rose, they also appear in "Family Possessions" (2016)
If you enjoyed Terror Tales for: Helene Udy, they also appear in "Hamlet's Ghost" (2015)
If you enjoyed Terror Tales for: Felissa Rose, they also appear in "Bethany" (2017)
